# Opppgave 2 Datainnsamling 

In [1]:
import requests
import pandas as pd
from datetime import date, timedelta, datetime
from pandasql import sqldf

In [2]:
# Input parameterene
client_id = "aff7c34e-993d-4132-81bc-1df3e81d7868" 
station_id = "SN18700"  # Blindern, Oslo
elements = ["air_temperature", "relative_humidity", "wind_speed", "precipitation_amount"]
start_date = (datetime.now() - timedelta(days=365)).strftime("%Y-%m-%d")
end_date = datetime.now().strftime("%Y-%m-%d")


Funksjonen visualiserer_data henter informasjon om værstasjoner fra Frost API ved hjelp av brukerens klient-ID. Den sender en forespørsel til API-et for å få en oversikt over alle tilgjengelige stasjoner. Deretter filtrerer den ut og strukturerer relevante data, som stasjons-ID, navn og land, og konverterer dette til en pandas DataFrame. Til slutt filtreres kun de stasjonene som ligger i Norge, slik at vi får en oversikt over norske værstasjoner som kan brukes videre i datainnsamling eller analyse. Funksjonen returnerer en tabell (DataFrame) med informasjon om disse norske stasjonene.

In [3]:
def visualiserer_data(client_id):

    sources_url = "https://frost.met.no/sources/v0.jsonld"
    try:
        response = requests.get(sources_url, auth=(client_id, ""), timeout=10)
        response.raise_for_status()
        sources_data = response.json()

    except requests.exceptions.RequestException as err:
        print(f"Feil ved innhenting av stasjon data: {err}")
        return None

    stations = []
    for source in sources_data["data"]:
        if "name" in source and "id" in source:
            stations.append({
                "id": source["id"],
                "name": source["name"],
                "country": source.get("country", "")
            })
    
    stations_df = pd.DataFrame(stations)
    norway_stations = stations_df[stations_df["country"] == "Norge"]
    
    if norway_stations.empty:
        print("No Norwegian stations found.")
        return None
    
    return norway_stations

visualiserer_data(client_id)

,id,name,country
0,SN47230,ÅKRA UNGDOMSSKOLE,Norge
1,SN20952,STATFJORD C,Norge
2,SN23670,E16 RYFOSS,Norge
3,SN59450,STADLANDET,Norge
4,SN55000,LUSTER - ORNES,Norge
...,...,...,...
2122,SN97120,E6 BARTA,Norge
2123,SN61062,TOMREFJORD,Norge
2126,SN15270,JUVVASSHØE,Norge
2128,SN74780,NAMSVATN II,Norge


Funksjonen get_station_id_by_name tar en klient-ID og et søkebegrep som input for å finne en værstasjon basert på navnet. Den henter først en liste over alle tilgjengelige stasjoner i Norge ved hjelp av funksjonen visualiserer_data. Deretter søker den etter stasjoner som har et navn som inneholder søkebegrepet, uavhengig av store og små bokstaver. Hvis én matchende stasjon finnes, returneres dens ID. Hvis flere stasjoner matcher, vises en liste med disse stasjonene, og den første stasjonen velges som standard. Hvis ingen stasjoner matcher, får brukeren beskjed om at ingen resultater ble funnet.

In [ ]:
def get_station_id_by_name(client_id, search_name):
    stations = visualiserer_data(client_id)
    if stations is None or stations.empty:
        print("Fant ingen stasjoner.")
        return None

    # Søker etter stasjoner med navn som inneholder søketeksten
    mask = stations["name"].str.lower().str.contains(search_name.lower())
    matches = stations[mask]

    if matches.empty:
        print(f"Ingen stasjoner matchet {search_name}")
        return None

    if len(matches) == 1:
        station_id = matches.iloc[0]["id"]
        print(f"Fant stasjon: {matches.iloc[0]['name']} → ID: {station_id}")
        
    else:
        print("Flere mulige stasjoner funnet:")
        print(matches[["id", "name"]])
        # Velger første som default så noe blir gjort
        station_id = matches.iloc[0]["id"]
        print(f"Velger den første: {station_id}")

get_station_id_by_name(client_id, "Oslo - Blindern")

Flere mulige stasjoner funnet:
           id                      name
18    SN18700           OSLO - BLINDERN
1365  SN18701       OSLO - BLINDERN PLU
1924  SN18703  OSLO - BLINDERN TESTFELT
Velger den første: SN18700


Funksjonen fetch_hourly_weather_to_csv brukes til å hente timesoppløste værdata fra Meteorologisk institutts Frost API, basert på en spesifikk stasjon, dato-intervall og ønskede værparametere. Den henter data for lufttemperatur, relativ luftfuktighet, middelvind og nedbør, og lagrer resultatene i en CSV-fil. For hver av de valgte værparametrene defineres passende elementkoder som benyttes i forespørslene til API-et. Dataene behandles og struktureres i en felles datastruktur før de lagres i fil.

I forsøket på å hente timesbaserte nedbørsdata fra Frost API fikk vi en "412 Precondition Failed"-feil. Dette tror vi skyldes at Meteorologisk institutt ikke benytter seg av en timeoppløsning for nedbør på den valgte stasjonen. Vi testet med flere stasjoner og fikk fortsatt ikke tak i nedbørsdata i timesoppløsning, og antar derfor at Meteorologisk institutt kun leverer nedbør på døgnbasis. I tillegg må man bruke sum når man henter ut nedbørsdata, da nedbør registreres som en akkumulert verdi over tid. For de øvrige variablene som lufttemperatur, relativ luftfuktighet og middelvind benyttes derimot mean, ettersom disse verdiene representerer gjennomsnitt i løpet av en gitt time. Det er likevel verdt å merke seg at kolonnen for nedbør fortsatt eksisterer i CSV-filen, men den inneholder ingen verdier. Dette gir fleksibilitet for fremtidig utfylling dersom nedbørsdata i ønsket oppløsning skulle bli tilgjengelig.

In [ ]:

def fetch_hourly_weather_to_csv(client_id, station_id, station_name, start_date, end_date, filename="../../data/weather_data.csv"):
    elements = {
        "Lufttemperatur": "air_temperature",
        "Relativ luftfuktighet": "relative_humidity",
        "Nedbør": "sum(precipitation_amount P1H)", 
        "Middelvind": "wind_speed",
    }

    base_url = "https://frost.met.no/observations/v0.jsonld"
    full_data = {}

    for label, element in elements.items():
        params = {
            "sources": station_id,
            "elements": element,
            "referencetime": f"{start_date}/{end_date}",
            "timeoffsets": "default",
            "timeresolutions": "PT1H"
        }
        
        # Tester om URL-en fungerer
        try:
            response = requests.get(base_url, params=params, auth=(client_id, ""), timeout=30) 
            response.raise_for_status()
            data = response.json().get("data", [])
            print(f"Hentet {label}: {len(data)} observasjoner") 
            if not data:
                print(f"Ingen data for {label} i perioden {start_date} til {end_date}")
        except requests.exceptions.RequestException as e:
            print(f"Feil ved henting av {label.lower()}: {e}")
            print(f"Responsstatus: {response.status_code if response else 'Ingen respons'}")
            continue

        for entry in data:
            time = pd.to_datetime(entry["referenceTime"]).tz_localize(None)
            value = entry["observations"][0]["value"]

            if time not in full_data:
                full_data[time] = {}
            full_data[time][label] = value

    # Konverterer til DataFrame
    records = []
    for time, values in sorted(full_data.items()):
        record = {
            "Navn": station_name,
            "Stasjon": station_id,
            "Tid (t)": time.strftime("%d.%m.%Y %H:00"),
            "Nedbør (mm)": values.get("Nedbør", ""),
            "Lufttemperatur (C)": values.get("Lufttemperatur", ""),
            "Middelvind (m/s)": values.get("Middelvind", ""),
            "Relativ luftfuktighet (g/m³)": values.get("Relativ luftfuktighet", "")
        }
        records.append(record)

    df = pd.DataFrame(records)
    df.to_csv(filename, sep=";", index=False, encoding="utf-8")
    print(f"Data lagret i: {filename}")


fetch_hourly_weather_to_csv(client_id, station_id, "Oslo - Blindern", start_date, end_date)

Hentet Lufttemperatur: 8760 observasjoner
Hentet Relativ luftfuktighet: 8760 observasjoner
Feil ved henting av nedbør: 412 Client Error: Precondition Failed for url: https://frost.met.no/observations/v0.jsonld?sources=SN18700&elements=sum%28precipitation_amount+P1H%29&referencetime=2024-04-20%2F2025-04-20&timeoffsets=default&timeresolutions=PT1H
Responsstatus: Ingen respons
Hentet Middelvind: 8760 observasjoner
Data lagret i: ../../data/weather_data.csv


### For å forstå dataens struktur og innhold har vi benyttet oss av diverse funksjoner 
- Vi startet med å bruke list comprehensions for å trekke ut spesifikke verdier, som for eksempel temperaturene, direkte fra DataFrame-en på en effektiv måte. Et eksempel på dette er funksjonen hent_varme_dager, som returnerer en liste over tidspunkt og temperatur for alle observasjoner der temperaturen overstiger en gitt grense.
- Deretter tok vi i bruk en iterator med df.iterrows() for å bla gjennom DataFrame-radene én etter én. Dette gjør det mulig å hente ut og presentere flere kolonneverdier samtidig på en oversiktlig måte. Vi laget en funksjon vis_observasjoner, som skriver ut stasjonsnavn, tidspunkt og relativ fuktighet for de første observasjonene i datasettet.
- Til slutt benyttet vi pandasql for å filtrere og analysere datasettet ved hjelp av SQL-syntaks direkte på DataFrame-en. Dette gjør det enklere å trekke ut spesifikke verdier basert på betingelser, som for eksempel alle tilfeller der middelvinden overstiger 10 m/s. Vi laget funksjonen hent_vind_over_grense for å gjøre dette på en fin måte.

In [ ]:
# Henter ut varme dager med list comprehension
def hent_varme_dager(df, temperaturgrense):
    
    varme_dager = [(rad["Tid (t)"], rad["Lufttemperatur (C)"]) 
                   for i, rad in df.iterrows() 
                   if rad["Lufttemperatur (C)"] > temperaturgrense]
    return varme_dager

df = pd.read_csv("../../data/weather_data.csv", sep=";", encoding="utf-8")
varme_dager = hent_varme_dager(df,temperaturgrense=27)
print(varme_dager[:10]) 


[('27.06.2024 15:00', 27.2)]


In [ ]:
# Itterer gjennom DataFrame og skriver ut de første 5 observasjonene
def vis_observasjoner(df, antall=5):
    print(f"Viser de første {antall+1} observasjonene:\n")
    for index, row in df.iterrows():
        print(f"Stasjon: {row['Stasjon']}, Tid: {row['Tid (t)']}, Relativ Fuktighet: {row['Relativ luftfuktighet (g/m³)']}(g/m³)")
        if index == antall:
            break

vis_observasjoner(df, antall=5)

Viser de første 6 observasjonene:

Stasjon: SN18700, Tid: 20.04.2024 00:00, Relativ Fuktighet: 90(g/m³)
Stasjon: SN18700, Tid: 20.04.2024 01:00, Relativ Fuktighet: 93(g/m³)
Stasjon: SN18700, Tid: 20.04.2024 02:00, Relativ Fuktighet: 93(g/m³)
Stasjon: SN18700, Tid: 20.04.2024 03:00, Relativ Fuktighet: 92(g/m³)
Stasjon: SN18700, Tid: 20.04.2024 04:00, Relativ Fuktighet: 92(g/m³)
Stasjon: SN18700, Tid: 20.04.2024 05:00, Relativ Fuktighet: 88(g/m³)


In [ ]:
# Pandasql for å hente ut data
def sterk_vind(df, grense):
    
    df = df.rename(columns={
        "Middelvind (m/s)": "middelvind",
        "Tid (t)": "tid"
    })

    vind = f"SELECT tid, middelvind FROM df WHERE middelvind > {grense}"
    result = sqldf(vind)
    return result

sterk_vind(df, grense=10)


,tid,middelvind
0,02.12.2024 18:00,10.7


# Oppgave 3 Databehandling 
- Funksjonen datafiltrering leser inn værdata fra en CSV-fil, konverterer tidskolonnen til riktig datoformat, sørger for at numeriske kolonner har riktig datatype (float), og fyller inn eventuelle manglende verdier med kolonnegjennomsnittet. Dette sikrer at datasettet er strukturert og klart for videre analyse.
- Funksjonen ekstremVerdier fjerner unormale eller ekstreme verdier i datasettet ved å filtrere temperaturer utenfor området -50 til 50 grader Celsius og vindhastigheter utenfor området 0 til 115 m/s. Dette bidrar til å sikre at dataene er realistiske og pålitelige for videre analyse.
- Funksjonen manglendeData håndterer manglende verdier i datasettet ved å bruke forskjellige metoder, avhengig av hva som er valgt:
    - "mean": Fyller inn manglende verdier med gjennomsnittet for den aktuelle kolonnen.
    - "median": Fyller inn med medianverdien for kolonnen.
    - "zero": Fyller inn med 0 for manglende verdier.

In [ ]:
# Datafiltrering og rensing
def datafiltrering(filepath):
    try:
        df = pd.read_csv(filepath, delimiter=";",encoding="utf-8")
        
        df["Tid (t)"] = pd.to_datetime(df["Tid (t)"], format="%d.%m.%Y %H:%M", errors="coerce")

        num_cols = ["Nedbør (mm)", "Lufttemperatur (C)", "Middelvind (m/s)", "Relativ luftfuktighet (g/m³)"]
        df[num_cols] = df[num_cols].astype(float)

        df.fillna(df.mean(numeric_only=True), inplace=True)

    except FileNotFoundError:
        print(f"Filen {filepath} ble ikke funnet.")
        return None

    return df

filepath = "../../data/weather_data.csv"
df_cleaned = datafiltrering(filepath)
print(df_cleaned.head())

              Navn  Stasjon             Tid (t)  Nedbør (mm)  \
0  Oslo - Blindern  SN18700 2024-04-20 00:00:00          NaN   
1  Oslo - Blindern  SN18700 2024-04-20 01:00:00          NaN   
2  Oslo - Blindern  SN18700 2024-04-20 02:00:00          NaN   
3  Oslo - Blindern  SN18700 2024-04-20 03:00:00          NaN   
4  Oslo - Blindern  SN18700 2024-04-20 04:00:00          NaN   

   Lufttemperatur (C)  Middelvind (m/s)  Relativ luftfuktighet (g/m³)  
0                 0.6               2.1                          90.0  
1                 0.1               1.0                          93.0  
2                -0.6               2.6                          93.0  
3                -1.0               2.1                          92.0  
4                -1.1               1.1                          92.0  


In [ ]:
# Fjern ekstreme verdier
def ekstremVerdier(df):
    df = df[(df["Lufttemperatur (C)"].between(-50, 50))] 
    df = df[(df["Middelvind (m/s)"].between(0, 115))] 
    
    return df

df_cleaned = ekstremVerdier(df_cleaned)


In [ ]:
# Tar i bruk 3 metoder for å fylle inn manglende data
def manglendeData(df, method="mean"):

    if method == "mean":
        df.fillna(df.mean(numeric_only=True), inplace=True)
    elif method == "median":
        df.fillna(df.median(numeric_only=True), inplace=True)
    elif method == "zero":
        df.fillna(0, inplace=True)
    
    return df

df_cleaned = manglendeData(df_cleaned, method="mean")


              Navn  Stasjon             Tid (t)  Nedbør (mm)  \
0  Oslo - Blindern  SN18700 2024-04-20 00:00:00          NaN   
1  Oslo - Blindern  SN18700 2024-04-20 01:00:00          NaN   
2  Oslo - Blindern  SN18700 2024-04-20 02:00:00          NaN   
3  Oslo - Blindern  SN18700 2024-04-20 03:00:00          NaN   
4  Oslo - Blindern  SN18700 2024-04-20 04:00:00          NaN   

   Lufttemperatur (C)  Middelvind (m/s)  Relativ luftfuktighet (g/m³)  
0                 0.6               2.1                          90.0  
1                 0.1               1.0                          93.0  
2                -0.6               2.6                          93.0  
3                -1.0               2.1                          92.0  
4                -1.1               1.1                          92.0  
